In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from nltk.corpus import stopwords

# Ajustes de exibição
pd.set_option('display.max_columns', None)

In [ ]:
# utilizar nível da vaga
# Adicionar etapa de limpeza na coluna título da vaga (remover traços, barras, etc)

In [4]:
# Lista de funções
def print_all(lista):

    print([*lista])

def encontrar_trechos(df: pd.core.frame.DataFrame,coluna: str, texto: str, usa_regex: bool = False):
    return df[coluna][df[coluna].str.contains(texto,regex = usa_regex, case=False).fillna(False)]

def encontrar_logica(df,coluna,texto,usa_regex = False):
    return list(df[coluna].str.contains(texto,regex = usa_regex, case=False).fillna(False))

def encontrar_chaves(serie, texto,usa_regex = False):
    return serie[serie.index.str.contains(texto, regex = usa_regex, case= False)]

def palavras_associadas(dados, coluna, regex):
    observado_completo = ' '.join(encontrar_trechos(dados,coluna,regex,usa_regex=True).values)
    lista = [word for word in observado_completo.split()]
    contagem = Counter(lista).most_common()
    serie = pd.Series(dict(contagem))
    return contagem, serie

In [5]:
# Carregar os arquivos Parquet
applicants = pd.read_parquet("applicants.parquet")
prospects = pd.read_parquet("prospects.parquet")
vagas = pd.read_parquet("vagas.parquet")

In [6]:
# Renomear colunas para evitar conflitos e identificar origem
applicants = applicants.add_prefix("applicants_")
prospects = prospects.add_prefix("prospects_")
vagas = vagas.add_prefix("vagas_")

In [76]:
# Primeiro merge: prospects + vagas
merged = pd.merge(
    prospects,
    vagas,
    left_on="prospects_vaga_id",
    right_on="vagas_id_vaga",
    how="inner"
)

# Segundo merge: resultado + applicants
final_df = pd.merge(
    merged,
    applicants,
    left_on="prospects_codigo",
    right_on="applicants_infos_basicas_codigo_profissional",
    how="inner"
)

In [77]:
# Filtra casos de MATCHES e NÃO MATCHES com base em valores selecionados
final_df = final_df[final_df['prospects_situacao_candidado'].isin(["Não Aprovado pelo Cliente",
                                                                   "Contratado pela Decision",
                                                                   "Não Aprovado pelo RH",
                                                                   "Não Aprovado pelo Requisitante",
                                                                   "Sem interesse nesta vaga",
                                                                   "Contratado como Hunting",
                                                                   "Aprovado",
                                                                   "Recusado",
                                                                   "Proposta Aceita"])]

final_df

,prospects_vaga_id,prospects_titulo,prospects_modalidade,prospects_quantidade_prospects,prospects_nome,prospects_codigo,prospects_situacao_candidado,prospects_data_candidatura,prospects_ultima_atualizacao,prospects_comentario,prospects_recrutador,vagas_id_vaga,vagas_data_requicisao,vagas_limite_esperado_para_contratacao,vagas_titulo_vaga,vagas_vaga_sap,vagas_cliente,vagas_solicitante_cliente,vagas_empresa_divisao,vagas_requisitante,vagas_analista_responsavel,vagas_tipo_contratacao,vagas_prazo_contratacao,vagas_objetivo_vaga,vagas_prioridade_vaga,vagas_origem_vaga,vagas_superior_imediato,vagas_nome,vagas_telefone,vagas_pais,vagas_estado,vagas_cidade,vagas_bairro,vagas_regiao,vagas_local_trabalho,vagas_vaga_especifica_para_pcd,vagas_faixa_etaria,vagas_horario_trabalho,vagas_nivel profissional,vagas_nivel_academico,vagas_nivel_ingles,vagas_nivel_espanhol,vagas_outro_idioma,vagas_areas_atuacao,vagas_principais_atividades,vagas_competencia_tecnicas_e_comportamentais,vagas_demais_observacoes,vagas_viagens_requeridas,vagas_equipamentos_necessarios,vagas_valor_venda,vagas_valor_compra_1,vagas_valor_compra_2,vagas_data_inicial,vagas_data_final,vagas_habilidades_comportamentais_necessarias,vagas_nome_substituto,vagas_categoria_contratacao,applicants_id_applicants,applicants_telefone_recado,applicants_telefone,applicants_infos_basicas_objetivo_profissional,applicants_infos_basicas_data_criacao,applicants_infos_basicas_inserido_por,applicants_infos_basicas_email,applicants_infos_basicas_local,applicants_infos_basicas_sabendo_de_nos_por,applicants_infos_basicas_data_atualizacao,applicants_infos_basicas_codigo_profissional,applicants_infos_basicas_nome,applicants_data_aceite,applicants_nome,applicants_cpf,applicants_fonte_indicacao,applicants_email,applicants_email_secundario,applicants_data_nascimento,applicants_telefone_celular,applicants_sexo,applicants_estado_civil,applicants_pcd,applicants_endereco,applicants_skype,applicants_url_linkedin,applicants_facebook,applicants_titulo_profissional,applicants_area_atuacao,applicants_conhecimentos_tecnicos,applicants_certificacoes,applicants_outras_certificacoes,applicants_remuneracao,applicants_nivel_profissional,applicants_nivel_academico,applicants_nivel_ingles,applicants_nivel_espanhol,applicants_outro_idioma,applicants_instituicao_ensino_superior,applicants_cursos,applicants_ano_conclusao,applicants_download_cv,applicants_qualificacoes,applicants_experiencias,applicants_outro_curso,applicants_id_ibrati,applicants_email_corporativo,applicants_cargo_atual,applicants_projeto_atual,applicants_cliente,applicants_unidade,applicants_data_admissao,applicants_data_ultima_promocao,applicants_nome_superior_imediato,applicants_email_superior_imediato,applicants_cv_pt,applicants_cv_en
2,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,2,Sra. Yasmin Fernandes,25364,Contratado pela Decision,2021-03-17,2021-04-12,Data de Inicio: 12/04/2021,Juliana Cassiano,4531,2021-03-10,NaT,2021-2607395-PeopleSoft Application Engine-Dom...,Não,Gonzalez and Sons,Valentim Duarte,Decision São Paulo,Vitória Melo,Srta. Bella Ferreira,PJ/Autônomo,,Contratação,Média: Média complexidade 6 a 10 dias,Nova Posição,Superior Imediato:,,,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Médio Completo,Nenhum,Nenhum,,Gestão e Alocação de Recursos de TI-,Key skills required for the job are:\n\nPeople...,O recurso Peoplesoft tem como responsabilidade...,"Remoto DEPOIS PRESENCIAL, TEMPO INDETERMINADO",,Notebook padrão -,0.0,hora,,NaT,NaT,,,PJ/Autônomo,25364,,(21) 96332-6963,Liderança / Desenvolvimento,17-03-2021 09:41:10,Juliana Cassiano,sra._yasmin_fernandes@hotmail.com,"São Paulo, São Paulo",Site de Empregos,12-04-2021 12:15:57,25364,Sra. Yasmin Fernandes,Cadastro anterior ao registro de aceite,Sra. Yasmin Fernandes,,Site de Empregos: APINFO / Linkedin,sra._yasmin_fernandes@hotmail.com,,31-01-1973,(21) 96332-6963,Masculino,Casado,Não,são paulo,,,,Liderança / Desenvolvimento,TI - Projetos,"- PeopleSoft (PeopleTools 8.49

In [136]:
features_palavras_chaves = {
                 'key_SAP': r'SAP',
                 'key_SD': r'\bSD\b',
                 'key_MM': r'\bMM\b',
                 'key_ABAP': r'\bABAP\b',
                 'key_AMS': r'AMS',
                 'key_JAVA': r'java',
                 'key_ORACLE': r'oracle',
                 'key_CLOUD': r'cloud|AWS|Azure',
                 'key_EBS': r'\bEBS\b',
                 'key_DBA': r'\bDBA\b',
                 'key_PROXXI': r'\bPROXXI\b',
                 'key_C_HASH': r'C#',
                 'key_OPERATIONS': r'Operations|Operações',
                 'key_PMO': r'\bPMO\b',
                 'key_ADM': r'\b(Adm|Administ|Administrativo|Administrador)\b',
                 'key_MARKETING': r'market',
                 'key_TI': r'\bTI\b',
                 'key_SALESFORCE': r'Salesforce',
                 'key_PROJETOS': r'Projetos',
                 'key_DADOS': r'Dados|Data',
                 'key_SECURITY': r'\b(Segurança|Security|Cyber)\b',
                 'key_WEB': r'\bWeb\b',
                 'key_SCRUM': r'\bScrum\b',
                 'key_SERVICE': r'Service',
                 'key_REACT_NATIVE_ANGULAR': r'\b(react|native|angular)',
                 'key_NET': r'.Net',
                 'key_Analista': r'Analista|Analyst',
                 'key_DEVOPS': r'devops',
                 'key_PYTHON': r'\bpython\b',
                 'key_C_PLUS_PLUS': r'C\+\+',
                 #'key_DEV': r'(Desenvolvedor|Developer|Dev)',
                 'key_COBOL': r'Cobol',
                 'key_ANDROID': r'Android',
                 'key_SQL': r'SQL',
}

In [137]:
# Criação de colunas binárias que informam palavras chave ou não
for coluna, regex_pattern in features_palavras_chaves.items():
    final_df[coluna] = final_df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)

C:\Users\renan_dyoksic\AppData\Local\Temp\ipykernel_3044\3250021381.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  final_df[coluna] = final_df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)
C:\Users\renan_dyoksic\AppData\Local\Temp\ipykernel_3044\3250021381.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  final_df[coluna] = final_df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)
C:\Users\renan_dyoksic\AppData\Local\Temp\ipykernel_3044\3250021381.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  final_df[coluna] = final_df['prospects_titulo'].str.contains(regex_pattern, regex= True, case= False).astype(int)


In [79]:
# Total de prospecções que sobraram após filtro de Matches
final_df.shape

(8173, 114)

In [138]:
# Total de prospecções que não teve nenhum valor "1" nas colunas binárias criadas
final_df[final_df[list(features_palavras_chaves.keys())].eq(0).all(axis=1)].shape

(1879, 148)

In [139]:
# Substitui df por apenas as linhas com "0" em todas as colunas binárias
final_df = final_df[final_df[list(features_palavras_chaves.keys())].eq(0).all(axis=1)]

In [ ]:
# SAP tem sub opções de consultor, SD, MM, ABAP, AMS
# Java
# Oracle possui subs de : Cloud, EBS, DBA, SQL
# PROXXI subs: C#
# Operations subs: PMO
# Adm. subs: Administrativo, Administrador
# Marketing
# TI subs: 
# Salesforce
# Projetos
# Dados: subs: Cientista
# Segurança subs: Cyber, Security
# Web
# Scrum
# Service
# React, native, ou angular
# Analista
# DevOps
# Python
# C++
# Desenvolvedor ou Developer ou Dev
# Cobol
# Android
# SQL

In [140]:
# Conta palavras mais repetidas em toda a coluna de título da prospecção
texto = " ".join(final_df['prospects_titulo'].dropna()).split()
Counter(texto)

Counter({'-': 764,
         'de': 250,
         'Consultor': 213,
         'Desenvolvedor': 195,
         'Developer': 97,
         'Senior': 94,
         'Dev': 78,
         'Pleno': 65,
         '/': 63,
         'End': 62,
         'Sênior': 57,
         'Engineer': 55,
         'QA': 54,
         'Suporte': 53,
         'Sr': 53,
         'Tech': 49,
         'Product': 43,
         'Owner': 43,
         'Full': 41,
         'SR': 41,
         'Front': 40,
         'Manager': 40,
         'Técnico': 40,
         'Quality': 38,
         'Arquiteto': 37,
         'Lead': 36,
         'Stack': 36,
         '–': 36,
         'e': 35,
         'Assistente': 35,
         'Gerente': 35,
         'Especialista': 34,
         'Software': 34,
         'Junior': 31,
         'Test': 30,
         'PL': 30,
         'Frontend': 30,
         'Telecom': 30,
         'Automation': 29,
         'Backend': 29,
         'Estoque': 28,
         'Power': 27,
         'Assurance': 26,
         'JDE': 24

In [135]:
# Contagem de palavras que estejam em linhas que possuem a palavra declarada para a função
palavras_associadas(final_df, 'prospects_titulo', 'Data')

([('Data', 95),
  ('-', 63),
  ('Big', 21),
  ('Engineer', 20),
  ('Desenvolvedor', 16),
  ('/', 15),
  ('(I)', 15),
  ('Engineering', 12),
  ('Consultor', 11),
  ('Data&AI', 9),
  ('science', 8),
  ('RPA', 8),
  ('13292255', 8),
  ('11723180', 6),
  ('Scientist', 6),
  ('Datastage', 5),
  ('Business', 5),
  ('Intelligence', 5),
  ('Science', 4),
  ('Practitioner', 4),
  ('Lake', 4),
  ('&', 4),
  ('Network', 4),
  ('Nível', 4),
  ('Pl.', 4),
  ('Sr.Porto', 4),
  ('Seguro', 4),
  ('SR', 4),
  ('RE-232128_1', 4),
  ('DATA', 3),
  ('Platform', 3),
  ('Manipulation', 3),
  ('GCP', 3),
  ('(ID', 3),
  ('25641):', 3),
  ('Pleno', 3),
  ('Senior', 3),
  ('Consultant', 3),
  ('Digital', 3),
  ('11262713', 3),
  ('JR', 3),
  ('BIG', 2),
  ('Support', 2),
  ('Migration', 2),
  ('11969150', 2),
  ('Engineer-Business', 2),
  ('(Z)', 2),
  ('de', 2),
  ('4252913', 2),
  ('Inteligence', 2),
  ('11582865', 2),
  ('Sr', 2),
  ('11691277', 2),
  ('Platforms-AWS', 2),
  ('LATAM', 2),
  ('MASTERDATA', 2